In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("D:\Datasets\online_retail.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.dropna(subset=['CustomerID'],inplace=True)


In [5]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [6]:
df['CustomerID']=df['CustomerID'].astype(int)
print(df['CustomerID'])

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 406829, dtype: int64


In [7]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate']

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 406829, dtype: datetime64[ns]

In [8]:
a=0
for i in  df['Quantity']:
    if i<0:
        a=a+1
print("Numbe of rows with negativity Quantity:",a)

Numbe of rows with negativity Quantity: 8905


In [9]:
df=df[(df['Quantity']>0)].copy()

In [10]:
df['TotalPrice']=df['Quantity']*df['UnitPrice']


In [11]:
reference_date=df['InvoiceDate'].max()+pd.Timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [12]:
rfm=df.groupby('CustomerID').agg({
    'InvoiceDate':lambda x: (reference_date - x.max()).days,
    'InvoiceNo':'count',
    'TotalPrice':'sum'})

In [13]:
rfm.rename(columns={
    'InvoiceDate':'Recency',
    'InvoiceNo':'Frequency',
    'TotalPrice':'Monetary'},inplace=True)
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,326,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,19,73,1757.55
12350,310,17,334.40


In [14]:
r_labels = range(5, 0, -1)
f_labels = range(1, 6)     
m_labels = range(1, 6)     

In [16]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=r_labels)

In [18]:
rfm['F_Score']=pd.qcut(rfm['Frequency'].rank(method='first'),q=5,labels=f_labels)

In [23]:
rfm['M_Score']=pd.qcut(rfm['Monetary'],q=5,labels=m_labels)

In [24]:
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

In [30]:
rfm['RFM_Score']=rfm[['R_Score','F_Score','M_Score']].sum(axis=1)
rfm.tail()

,Recency,Frequency,Monetary,R_Score,F_Score,M_score,M_Score,RFM_Segment,RFM_Score
CustomerID,,,,,,,,,
18280,278,10,180.60,1,1,1,1,111,3
18281,181,7,80.82,1,1,1,1,111,3
18282,8,12,178.05,5,1,1,1,511,7
18283,4,756,2094.88,5,5,5,5,555,15
18287,43,70,1837.28,3,4,4,4,344,11


In [31]:
def segment_customer(df):
    if df['RFM_Score'] >= 13: # Top scores
        return 'Champions'
    elif df['RFM_Score'] >= 10:
        return 'Loyal Customers'
    elif df['RFM_Score'] >= 7:
        return 'Potential Loyalists'
    elif df['RFM_Score'] >= 5:
        return 'At Risk' 
    else:
        return 'Hibernating' 

In [32]:
rfm['Customer_Segment'] = rfm.apply(segment_customer, axis=1)

In [33]:
print(rfm[['Recency', 'Frequency', 'Monetary', 'Customer_Segment']].head())

            Recency  Frequency  Monetary     Customer_Segment
CustomerID                                                   
12346           326          1  77183.60  Potential Loyalists
12347             2        182   4310.00            Champions
12348            75         31   1797.24  Potential Loyalists
12349            19         73   1757.55      Loyal Customers
12350           310         17    334.40              At Risk


In [37]:
rfm.to_csv('RFM_Analysis.csv')